# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298316777617                   -0.85    5.5
  2   -8.300189308800       -2.73       -1.25    1.0
  3   -8.300431529925       -3.62       -1.88    2.0
  4   -8.300462334030       -4.51       -2.77    2.6
  5   -8.300464295174       -5.71       -3.11    2.6
  6   -8.300464463684       -6.77       -3.26    4.5
  7   -8.300464549627       -7.07       -3.41    7.4
  8   -8.300464600679       -7.29       -3.56    1.9
  9   -8.300464630617       -7.52       -3.74    1.5
 10   -8.300464636201       -8.25       -3.84    1.9
 11   -8.300464642606       -8.19       -4.09    2.0


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64187546707                   -0.70    6.0
  2   -16.67860945194       -1.43       -1.14    1.4
  3   -16.67920725545       -3.22       -1.88    3.4
  4   -16.67927905346       -4.14       -2.74    2.2
  5   -16.67928601691       -5.16       -3.10    5.2
  6   -16.67928620515       -6.73       -3.54    2.4
  7   -16.67928622153       -7.79       -4.00    2.0


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32553659968                   -0.56    6.6
  2   -33.33271772080       -2.14       -1.00    1.0
  3   -33.33408954994       -2.86       -1.74    4.8
  4   -33.33427581094       -3.73       -2.64    2.0
  5   -33.33577882580       -2.82       -2.40    4.9
  6   -33.33694221026       -2.93       -2.51    7.8
  7   -33.33694119001   +   -5.99       -2.53    3.2
  8   -33.33673217951   +   -3.68       -2.29    2.6
  9   -33.33666135781   +   -4.15       -2.23    1.5
 10   -33.33667993443       -4.73       -2.24    1.0
 11   -33.33672090566       -4.39       -2.28    2.5
 12   -33.33674077495       -4.70       -2.30    1.0
 13   -33.33676978980       -4.54       -2.33    1.0
 14   -33.33679106001       -4.67       -2.36    

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298545540384                   -0.85    5.8
  2   -8.300277441082       -2.76       -1.59    1.0
  3   -8.300440115771       -3.79       -2.63    1.5
  4   -8.300412105170   +   -4.55       -2.40    5.4
  5   -8.300464382737       -4.28       -3.43    1.1
  6   -8.300464573302       -6.72       -3.83    2.9
  7   -8.300464640431       -7.17       -4.34    1.1


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32247701807                   -0.56    7.6
  2   -33.23728538558   +   -1.07       -1.23    2.5
  3   +36.53359223221   +    1.84       -0.14    8.9
  4   -33.32034654629        1.84       -1.78    6.6
  5   -33.10573584868   +   -0.67       -1.34    4.0
  6   -32.70503967440   +   -0.40       -1.16    5.0
  7   -33.30443540248       -0.22       -1.68    4.6
  8   -33.33555870355       -1.51       -2.08    2.4
  9   -33.33671755390       -2.94       -2.53    2.2
 10   -33.33633357877   +   -3.42       -2.48    2.5
 11   -33.33692260423       -3.23       -3.04    2.1
 12   -33.33693263701       -5.00       -3.19    2.4
 13   -33.33693341267       -6.11       -3.35    1.8
 14   -33.33694354630       -4.99       -3.87    

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.